# Tarea 2

Integrantes: Gregorio Salaberry y Rocío Monsalve

- Usando las funciones anteriores, o desarrollando las propias, escriba una función que pueda valorizar, por simulación de Montecarlo, calls y puts sobre bonos cero cupón con el modelo de HW. Considere puts y calls de plazo máximo igual a 1Y.
- Su función debe poder recibir la semilla.
- Compare sus resultados con las fórmulas del modelo y muestre que diferencias obtiene usando 1000 simulaciones y una semilla predeterminada.
- Valorice también por simulación de Montecarlo el bono cupón cero a 1Y. Muestre que diferencias obtiene respecto al valor de mercado del bono usando 1000 simulaciones y una semilla predeterminada.

Se cargan las librerias necesarias para realizar la tarea:

In [36]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d
# from modules
import auxiliary as aux
from numpy import random as rnd
import plotly.express as px
import pandas as pd
import numpy as np
import math
from scipy.stats import norm

In [37]:
frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'} #sirve para formatear los dataframe que pondremos en pantalla

## Funciones de Notebooks anteriores

### Cargar Curva Cero Cupon e Interpolación

In [38]:
#Se cargan los datos para construir la curva cupon cero
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [39]:
#Se muestra la tabla
curva.head().style.format(frmt)

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


In [40]:
#Necesitamos tener derivadas para la función theta
#Se crean estas nuevas columnas en el datagrame de la curva
curva['t'] = curva['plazo'] / 365.0
curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)
curva.head().style.format(frmt)

,plazo,tasa,df,t,rate
0,1,0.0811%,99.999778%,0.002740,0.000811
1,7,0.0841%,99.998388%,0.019178,0.000841
2,14,0.0780%,99.997010%,0.038356,0.000780
3,21,0.0774%,99.995549%,0.057534,0.000774
4,33,0.0781%,99.992942%,0.090411,0.000781


In [41]:
#Construimos este nuevo objeto para poder interpolar la curva, zcurva es una función. 
#Se utiliza una interpolación cúbica, de manera de obtener una curva más suave
zcurva = interp1d(curva['t'],
                  curva['rate'],
                  kind='cubic',
                  fill_value="extrapolate")

### Derivadas para la función Zcurva

In [42]:
#Funciones de las derivadas 
def zrate(t: float) -> float:
    return zcurva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

#Probar las funciones realizadas
t = 2
print(f'zrate({t:.2f}) = {zrate(t):.4%}')
print(f'dzrate({t:.2f}) = {dzrate(t):.4%}')
print(f'd2zrate({t:.2f}) = {d2zrate(t):.4%}')

zrate(2.00) = 0.0568%
dzrate(2.00) = -0.0066%
d2zrate(2.00) = 0.0584%


Podemos ahora definir las funciones `fwd(0,t)` y `dfwd(0,t)`.

### Funciones `fwd(0,t)` y `dfwd(0,t)`

In [43]:
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [44]:
#Probar las funciones realizadas
print(f'fwd({t:.2f}) = {fwd(t):.4%}')
print(f'dfwd({t:.2f}) = {dfwd(t):.4%}')

fwd(2.00) = 0.0436%
dfwd(2.00) = 0.1036%


### Función $\theta$

In [45]:
#Ahora se puede crear una función para obtener el Theta
sigma = .015
gamma = .5

# Proxy de la tasa instantánea r(t) es una tasa entre t y t + dt (donde dt es un intervalo infitesimal)
r0 = float(zrate(0))

#Plazos de t y T
t=1
T=2

In [46]:
#Se crea la función theta
def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

In [47]:
#Se prueba la función creada
print(f'theta({t:.2f}) = {theta(t):.4%}')

theta(1.00) = 0.0139%


# Valorizar Call o Put sobre Bono Cero Cupon 

Usando las funciones anteriores, o desarrollando las propias, escriba una función que pueda valorizar, por simulación de Montecarlo, calls y puts sobre bonos cero cupón con el modelo de HW. Considere puts y calls de plazo máximo igual a 1Y.

## Simulación MonteCarlo

In [48]:
#Se utiliza el código para crear la función de montecarlo
def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed = None):
    """
    """
    dt = 1 / 264.0
    num_steps += 1
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(0, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i * dt)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return tiempo, sim

### Tasa Corta

In [49]:
#Se realiza la simulación de MonteCarlo para obtener el valor de la tasa

num_sim = 1000
num_steps = 263
print(f"num_steps: {num_steps}")
dt=1/264.0
seed = 1234
df = 0

#Se realiza la simulacion de MC, utilizando la función que se encuentra en el punto anterior. 
tiempo, s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
for sim in s:
    df += math.exp(-dt * np.sum(sim))  #se calculan los df

    
ez = df / num_sim
z_curva = math.exp(-zrate(t) * t)
print(f"ez: {ez: .8%}")
print(f"z_curva: {z_curva: .8%}")


#  Extrae la Última Tasa de Cada Simulación
last_rates = [sim[-1] for sim in s]

num_steps: 263
ez:  99.95341796%
z_curva:  99.92978688%


### Precio Bono Cero Cupon 

In [50]:
#Función A de la fórmula para un bono cupón cero:

def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [51]:
#Función  𝐵 de la fórmula para un bono cupón cero:

def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - gamma: intensidad de reversión del modelo HW
    - t:
    - T:
    
    return:
    
    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [52]:
#Bono Cero Cupon 
def zero_hw(r: float, 
            gamma: float, 
            sigma: float,
            zrate: float, 
            fwd: float, t: float, T: float) -> float:
    
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

### Precio del Bono Cupon Cero

In [53]:
#Se obtienen los precios del bono cupon cero en t a T con las tasas simuladas anteriormente
t=1
T=2
#precios = [zero_hw(r, gamma, sigma, zrate, fwd, t, T) * 100 for r in last_rates]
precios = [zero_hw(r, gamma, sigma, zrate, fwd, t, T) for r in last_rates]
precios_df=pd.DataFrame()
precios_df["Precios"]=precios
precios_df.head(5)

,Precios
0,0.993345
1,1.002322
2,0.994522
3,1.008650
4,0.988016


### Cálculo del Payoff

In [54]:
#Formula para obtener el payoff de la call: max(math.log(max(z - strike, .0001)), .0001)
#Formula para obtener el payoff de la put: max(math.log(max(strike-k, .0001)), .0001)

In [55]:
# Se crea una nueva función que retorna el payoff

def Payoff_op(z:float, strike: float, opcion) -> float:
    
    """
    z: es el precio del bono cupón cero al vencimiento
    strike: es el strike de la opción
    opcion: corresponde a si es una call o put. Solamente recibe como parámetros "call"
    o "put"
    """
    if opcion=="call":
         return max((max(z - strike, .0001)), .0001)
    elif opcion== "put":
        return max((max(strike - z, .0001)), .0001)
    
    else:
        print("Debe escribir call o put")
        #esto es el caso en que no se recibe bien el input de la función, ya que la función está diseñada
        #solamente para recibir la palabra "call" o la palabra "put". En caso contrario, deberá imprirmir 
        #por pantalla que está incorrecto el argumento que entró a la función.

In [56]:
# Se prueba la funcion
#Eleccion de los strikes correspondientes. 
strikes = [0.98, 0.99, 1.00, 1.01, 1.02]

valor_payoffcall = {strike:[Payoff_op(z, strike, "call") for z in precios] for strike in strikes}

valor_payoffput=valor_call = {strike:[Payoff_op(z, strike, "put") for z in precios] for strike in strikes}


### Cálculo del Valor Presente

In [57]:
for sim in s:
    df = math.exp(-dt * np.sum(sim))  #se calculan los df
    valor_calls2= [df * np.average(valor_payoffcall[strike]) 
                  for strike in strikes]



In [58]:
valor_calls2

[0.019786913277178707,
 0.010386615009360304,
 0.003492364342275166,
 0.0006463731432217685,
 0.00012771747017441273]

In [59]:
t_ = 1.0 # 1 año que corresponde al vencimiento.

#Valor Presente de la Call
valor_calls= [math.exp(-t_ * float(zrate(t_))) * np.average(valor_payoffcall[strike]) for strike in strikes]
for v in valor_calls:
    print(f'valor call: {v:.8f}')

valor call: 0.01978758
valor call: 0.01038697
valor call: 0.00349248
valor call: 0.00064639
valor call: 0.00012772


In [60]:
#Valor Presente de la Put
valor_puts= [math.exp(-t_ * float(zrate(t_))) * np.average(valor_payoffput[strike]) for strike in strikes]
for v in valor_puts:
    print(f'valor put: {v:.8f}')

valor put: 0.00013782
valor put: 0.00070342
valor put: 0.00372750
valor put: 0.01080247
valor put: 0.02025484


In [61]:
#Se crea un dataframe para poder ver los resultados que se obtienen 
df_Resultados=pd.DataFrame()
df_Resultados["Strikes"]=strikes
df_Resultados["Call MC"]=valor_calls
df_Resultados["Put MC"]=valor_puts
df_Resultados

,Strikes,Call MC,Put MC
0,0.98,0.019788,0.000138
1,0.99,0.010387,0.000703
2,1.00,0.003492,0.003728
3,1.01,0.000646,0.010802
4,1.02,0.000128,0.020255


## Valorización de la Opción con las fórmulas del modelo

Compare sus resultados con las fórmulas del modelo y muestre que diferencias obtiene usando 1000 simulaciones y una semilla predeterminada. Las siguientes fórmulas son las que se utilizarán:

$$
\begin{equation}
Call\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(d_1\right)-KN\left(d_2\right)\right]
\end{equation}
$$


$$
\begin{equation}
d_1=\frac{\log\left(\frac{Z\left(r_0,0,T_B\right)}{KZ\left(r_0,0,T_O\right)}\right)+\frac{1}{2}S_Z\left(T_O\right)^2}{S_Z\left(T_O\right)}
\end{equation}
$$


$$
\begin{equation}
d_2=d_1-S_Z\left(T_O\right)
\end{equation}
$$


$$
\begin{equation}
S_Z\left(T_O\right)=B\left(T_O,T_B\right)\sqrt{\frac{\sigma^2}{2\gamma^*}\left(1-\exp\left(-2\gamma^* T_O\right)\right)}
\end{equation} 
$$

Y para una put:

$$
\begin{equation}
Put\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[KN\left(-d_2\right)-\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(-d_1\right)\right]
\end{equation}
$$

y los mismos valores anteriores para $d_1$ y $d_2$.

In [62]:
#Verifiquemos que los valores que entrega la fórmula de HW coinciden con los datos de la curva:
T0=1
TB=2

# Primero se obtienen los valores del precio de la curva cero cupon para ambos T.
#zero_hw(r, gamma, sigma, zrate,fwd: float, t: float, T: float) -> float:
r=zrate(0)
zero_T0=zero_hw(r, gamma, sigma, zrate, fwd, 0, T0)
zero_TB=zero_hw(r, gamma, sigma, zrate, fwd, 0, TB)

print(f'Precio T0         :  {zero_T0:.8f}')
print(f'Precio T0 de Curva:  {math.exp(-zrate(T0)*T0):.8f}')
print(f'Precio TB         :  {zero_TB:.8f}')
print(f'Precio TB de Curva:  {math.exp(-zrate(TB)*TB):.8f}')


Precio T0         :  0.99929787
Precio T0 de Curva:  0.99929787
Precio TB         :  0.99886549
Precio TB de Curva:  0.99886549


Los valores anteriores nos indican que la función zero_hw es correcta.

### Calculo Call y Put con Fórmulas

In [63]:
#Se crea una función para obtener los valores de call o puts con la fórmula cerrada

def formula_BS (zrate: float, 
                gamma: float, 
                sigma: float, 
                T0: float, 
                TB: float, 
                strike:float,
                opcion) -> float:
    
    #Se obtienen los valores que van en la ecuacion para obtener calls y puts
    zero_T0=math.exp(-zrate(T0)*T0)
    zero_TB=math.exp(-zrate(TB)*TB)
    
    if opcion=="call":
        epsilon=1
    elif opcion== "put":
        epsilon=-1
    else:
        print("Debe escribir call o put")
    
    #Se aplica la formula de Black Scholes
    Sz=b_hw(gamma,T0,TB)*math.sqrt((sigma**2/(2*gamma)*(1-math.exp(-2*gamma*T0))))
    d1=(math.log(zero_TB/(strike*zero_T0))+ 0.5*(Sz**2))/Sz
    d2=d1- Sz
    
    return epsilon* zero_T0*((zero_TB/zero_T0)*norm.cdf(epsilon*d1)- strike*norm.cdf(epsilon*d2))
 
#Es importante destacar que el valor del Payoff siempre debe ser positivo, ya que es el valor esperado
#de un número y el valor esperado de un número siempre es positivo. 

In [64]:
#Strikes que se utilizarán, se utilizan distintos valores de strikes para poder obtener los 
#valores de las opciones en diferentes casos.
strikes = [0.98, 0.99, 1.00, 1.01, 1.02]

#Es importante destacar que los strikes deben estar en la misma función en que se encuentra el bono.
#Si el bono están en función de 100, el strike debe estar igual. En nuestro caso, no se encuentran
#en función de 100. 


#Obtencion del valor de la call
valor_callBS= {strike:(formula_BS(zrate, gamma, sigma, T0, TB, strike, "call")) for strike in strikes}
df_1 = pd.DataFrame(list(valor_callBS.items()),columns = ['Strikes','Call FC']) 

#Obtencion del valor de la put
valor_BSput= {strike:(formula_BS(zrate, gamma, sigma, T0, TB, strike, "put")) for strike in strikes}
df_2 = pd.DataFrame(list(valor_BSput.items()),columns = ['Strikes','Put FC']) 


#Unirlos al mismo DataFrame
df_Resultados["Call FC"]=df_1["Call FC"]
df_Resultados["Put FC"]=df_2["Put FC"]

df_Resultados
#Se muestran los resultados obtenidos

,Strikes,Call MC,Put MC,Call FC,Put FC
0,0.98,0.019788,0.000138,0.019613,0.000059
1,0.99,0.010387,0.000703,0.010302,0.000741
2,1.00,0.003492,0.003728,0.003528,0.003961
3,1.01,0.000646,0.010802,0.000639,0.011064
4,1.02,0.000128,0.020255,0.000052,0.020471


In [75]:
# Test

import hull_white as hw

calls = [(r.Strikes, hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=r.Strikes,
    to=T0,
    tb=TB,
    r0=zrate(0),
    zo=math.exp(-zrate(T0) * T0),
    zb=math.exp(-zrate(TB) * TB),
    gamma=gamma,
    sigma=sigma)) for r in df_Resultados.itertuples()]

for c in calls:
    print(f'{c[0]:.2f}, {c[1]:.6f}')

print()

puts = [(r.Strikes, hw.zcb_call_put(
    hw.CallPut.PUT,
    strike=r.Strikes,
    to=T0,
    tb=TB,
    r0=zrate(0),
    zo=math.exp(-zrate(T0) * T0),
    zb=math.exp(-zrate(TB) * TB),
    gamma=gamma,
    sigma=sigma)) for r in df_Resultados.itertuples()]

for p in puts:
    print(f'{p[0]:.2f}, {p[1]:.6f}')

0.98, 0.019613
0.99, 0.010302
1.00, 0.003528
1.01, 0.000639
1.02, 0.000052

0.98, 0.000059
0.99, 0.000741
1.00, 0.003961
1.01, 0.011064
1.02, 0.020471


In [76]:
#Calculo de los errores relativos obtenidos:
df_Resultados["Error relativo Call"]=abs(df_Resultados["Call MC"]
                                      - df_Resultados["Call FC"])/df_Resultados["Call MC"] 

df_Resultados["Error relativo Put"]=abs(df_Resultados["Put MC"]
                                     - df_Resultados["Put FC"])/df_Resultados["Put MC"] 

df_Resultados

,Strikes,Call MC,Put MC,Call FC,Put FC,Error relativo Call,Error relativo Put
0,0.98,0.019788,0.000138,0.019613,0.000059,0.008846,0.572181
1,0.99,0.010387,0.000703,0.010302,0.000741,0.008192,0.053810
2,1.00,0.003492,0.003728,0.003528,0.003961,0.010281,0.062580
3,1.01,0.000646,0.010802,0.000639,0.011064,0.012185,0.024199
4,1.02,0.000128,0.020255,0.000052,0.020471,0.589859,0.010658


Al calcular los errores relativos obtenidos, podemos decir que las diferencias entre el valor de las opciones call que se obtuvo de la simulación de montecarlo y el valor que se obtuvo con la fórmula cerrada son muy pequeñas, por lo que los resultados obtenidos son bastante buenos. Ocurre lo mismo al calcular los valores de las opciones put. 

**ADV:** algunos errores relativos son muy altos, 0.5 es un 50%.

# Simulación Monte Carlo y Valor Mercado Bono Cero Cupon

Valorice también por simulación de Montecarlo el bono cupón cero a 1Y. Muestre que diferencias obtiene respecto al valor de mercado del bono usando 1000 simulaciones y una semilla predeterminada.

In [77]:
#Se realiza la simulación de MonteCarlo para el bono cupon cero a 1Y. Se utilizan 1000 simulaciones 

t=1
num_sim = 1000
num_steps = 263
print(f"num_steps: {num_steps}")
dt=1/264.0
seed = 2000

df = 0
tiempo, s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
for sim in s:
    df += math.exp(-dt * np.sum(sim))


ez = df / num_sim #resultado con simulacion de montecarlo

z_curva = math.exp(-zrate(t) * t)  #resultado de mercado
print(f"ez: {ez: .8}")
print(f"z_curva: {z_curva: .8}")

num_steps: 263
ez:  0.99929734
z_curva:  0.99929787


## Error Relativo Porcentual entre ambos resultados anteriores

In [78]:
error_porcentual= abs(ez-z_curva)/z_curva *100
print(f"Error relativo porcentual: {error_porcentual: .8%}%")

Error relativo porcentual:  0.00524558%%


Podemos observar que se obtuvo un errorr relativo porcentual pequeño al calcular por simulacion de MonteCarlo el bono cupon cero a 1 año.

In [79]:
for sim in s:
    df = math.exp(-dt * np.sum(sim))  #se calculan los df
    valor_bono= df * 1   #el payoff del bono cupon cero es igual a 1. 
valor_bono

1.0185658839228664

**AD:** no hay explicaciones, sólo 1 comparación.